# Data load and pre-processing

Imports

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'

import datetime
import sqlite3
import pandas as pd
# import matplotlib as mat
# from pylab import *
# import seaborn as sns
from sklearn.preprocessing import LabelEncoder

## Useful functions

In [3]:
def beatiful_coef(coefs, feature_names):
    return pd.DataFrame(coefs, index=feature_names, columns=['coef']).sort_values('coef', ascending=False)

def purpose_category(purpose):
    if purpose == '':
       return 'Empty'
    
    categories = [
        ('Business', ['бизнес', 'оборот', 'проект', 'реклам']),
        ('Perecredit', ['перекред', 'рекредит', 'рефинан', 'займ']),
        ('Personal', ['личн', 'нуж', 'расхо']),
        ('Salary', ['зарпл', 'з/п', 'зп']),
        ('Cryptocurrency', ['криптовал']),
        ('Purchase', ['оплат', 'закуп', 'покуп']),
    ]
    
    for rule in categories:
        category = rule[0]
        terms = rule[1]
        for term in terms:
           if term in purpose.lower():
                return category
    return 'Other'

def connotation(text):
    terms = ['много', 'всегда', 'лет', 'гарант', 'постоя', 'ни одн', 'просроч', 'стабил', '!', 'возвращ', 'давно', 'идеал']
    for term in terms:
        if term in text:
            return True
    return False

def score(row):
    # delinq_in_periods = (row['period'] + row['delinq']) / row['period']
    # delinq_in_periods = 1.0 if delinq_in_periods <= 0 else delinq_in_periods
    # score = 1 / delinq_in_periods
    score = row['period'] if row['period'] + row['delinq'] == 0 else row['period'] / (row['period'] + row['delinq'])
    return score if score < 1 else 1

## Load data

In [4]:
date_columns = ['deal_date', 'return_date', 'expiration_date', 'report_date']

conn = sqlite3.connect("../../data/loancwm_2019-03-12.db")
deals = pd.read_sql_query("select * from deals;", conn, parse_dates=date_columns)
conn.close()

# deals.info()

## Processing

### Filter out open positions

In [5]:
open_positions = \
    (deals['outstanding'] > 0) & \
    (deals['report_date'] < deals['expiration_date']) # + pd.DateOffset(years=1))

deals = deals[~open_positions].copy()

### Fixing data types

In [6]:
# Converting string value of pass_date to datetime type
deals['pass_date'] = pd.to_datetime(deals['pass_date'], format='%m/%d/%Y')
date_columns += ['pass_date']

deals['rating'] = deals['rating'] / 100

# Replace the default field text with empty string 
deals['purpose'] = deals.apply(lambda row: '' if row['purpose'].startswith('опишит') else row['purpose'], axis=1)
deals['guarantee'] = deals.apply(lambda row: '' if row['guarantee'].startswith('опишит') else row['guarantee'], axis=1)
deals['address'] = deals.apply(lambda row: '' if row['address'].startswith('представьтесь') else row['address'], axis=1)
deals['recommend'] = deals.apply(lambda row: '' if row['recommend'].startswith('есть ли у') else row['recommend'], axis=1)

### Adding new features

In [7]:
# Mapping the city name to city number code
lb_make = LabelEncoder()
deals['reg_city_code'] = lb_make.fit_transform(deals["reg_city"])

# Indicates that a loaner left the fields 'purpose', 'guarantee', 'address', 'recommend' empty
deals['empty_info'] = deals.apply(lambda row: 1 if (row['purpose'] == '') & (row['guarantee'] == '') & (row['address'] == '') & (row['recommend'] == '') else 0, axis=1)

# Classify the purpose into categories and mapping them to number codes
deals['purpose_category'] = deals.apply(lambda row: purpose_category(row['purpose']), axis=1)
lb_make = LabelEncoder()
deals['purpose_code'] = lb_make.fit_transform(deals["purpose_category"])

# deals['connotation'] = deals.apply(lambda row: connotation(row['purpose']) | connotation(row['guarantee']) | connotation(row['address']) | connotation(row['recommend']), axis=1)
# deals['connotation'] = deals.apply(lambda row: 1 if row['connotation'] else 0, axis=1)

# Extracting year and month from the datetime columns
for date_column in date_columns:
    deals[date_column + '_year'] = deals.apply(lambda row: row[date_column].year, axis=1)
    deals[date_column + '_month'] = deals.apply(lambda row: row[date_column].month, axis=1)
#    deals[date_column + '_m'] = deals.apply(lambda row: row[date_column].year * 100 + row[date_column].month, axis=1)

In [55]:
deals.groupby('purpose_category')['purpose'].count()

purpose_category
Business          1358
Cryptocurrency       3
Empty              998
Other              352
Perecredit         275
Personal          1174
Purchase           305
Salary              33
Name: purpose, dtype: int64

### Labeling

In [8]:
deals['score'] = deals.apply(lambda row: score(row), axis=1)
deals['give'] = deals.apply(lambda row: (row['outstanding'] == 0) & (row['score'] >= 0.2), axis=1)
# deals[['period', 'delinq', 'outstanding', 'score', 'give']]

In [15]:
counts = deals.groupby('give')['give'].count()
print(counts)
print(1 - counts[0] / counts[1])
print(counts[0], counts[1])


give
False     265
True     4233
Name: give, dtype: int64
0.93739664540515
265 4233


In [36]:
#for a in deals[deals['address'] != ''][['address']].iterrows():
#    print(a[1][0] + '\n')

In [46]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4498 entries, 0 to 4586
Data columns (total 51 columns):
tender_num                        4498 non-null int64
deal_date                         4498 non-null datetime64[ns]
report_date                       4498 non-null datetime64[ns]
period                            4498 non-null int64
credit_amount                     4498 non-null float64
return_amount                     4498 non-null float64
rate                              4498 non-null float64
outstanding                       4498 non-null float64
return_date                       4498 non-null datetime64[ns]
expiration_date                   4498 non-null datetime64[ns]
tender_credit_amount              4498 non-null float64
tender_return_amount              4498 non-null float64
tender_interest_rate              4498 non-null float64
credit_amount_average             4498 non-null float64
credit_amount_to_average_ratio    4498 non-null float64
average_repeated_loan_count  